First, import nesassary libraries:

In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation 
import numpy as np
import math
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

The separation energy is the minimum energy needed to remove a particle from a nucleus. In our model, we will define our own seperation energy as $S_p = S_0+\sum S_k(Z_0-Z)^k$. In the follow calculation, we will fix the neutron number N. if k is up to 2. So our separation energy will be $S_p = S_0 + S_1(Z_0-Z) + S_2(Z_0-Z)^2$. We can define our $S_p$ as below. Where x is the array $S_p$'s, while Z is the proton number.

In reality, we also need to add the pairing and shell effects. For pair effect: when Z is even, pair effect will add a constant value (delta) to x, otherwise deduct a delta. Similarly for shell effect: when Z is larger than the magic number (28, 50, 82 ...), the total sn function will decrease by a constant, which we can input its value to the *magic* array. 

In [2]:
def sp_main(x, Z, s_array, delta, magic):
    for i in range(len(s_array)):
        x += s_array[i] * np.power(Z[0] - Z, i)
        
    for i in range(len(x)):
        if (Z[i] % 2) == 0:
            x[i] += delta
        else:
            x[i] -= delta
            
    for i in range(len(magic)):
        x -= magic[i][0] * np.heaviside(Z - magic[i][1], 0)
    x = x + S_spike[0,:]
    return x

When we have the separation function, we can 
integrate it to get the (p, $\gamma$)-($\gamma$, p) equilibrium abundance ratio. So we will integrate $S_p$ over proton number $Z$, the useful results in the abundance equation are shown below.

In [3]:
def integral(Z, delta, magic):    
    Fp = []
    shell = np.zeros(len(Z))
    shell2 = np.zeros(len(Z))
    shell3 = np.zeros(len(Z))
    
    for n in range (len(Z)):
        fp = delta*(-1)**(Z[0]+1)*Pi(n-Z[0])
        Fp.append(fp)
    Pair1 = np.array(Fp)
    
    for i in range (len(magic)):
        shell += magic[i][0] * np.heaviside(Z - magic[i][1], 0)
        shell2 += magic[i][0] * (np.heaviside(Z - magic[i][1], 0))*(Z[0]-magic[i][1])
#        shell3 += magic[i][0] * (np.heaviside(N-70- magic[i][1], 0))*(N-magic[i][1])
        
    intgl = np.vstack((Pair1, shell, shell2)) 
    return intgl

def Pi(x):
    if (x % 2) == 0:
        return(0)
    else:
        return(1)

In order to get a movie, let's first create a figure with labels.

In [4]:
fig = plt.figure(figsize=(8,6))

ax1 = plt.subplot(121)

ax1.set_xlabel(r'Energy [MeV]')
ax1.set_ylabel('Proton number, Z')
ax1.set_ylim([40,110])
ax1.set_xlim([0,24])
txt_title = ax1.set_title('',fontsize=10)
line1, = ax1.plot([], [], label = r'-$\mu_p^{\prime}$')
line2, = ax1.plot([], [], label = r'$S_p$')
ax1.legend()

ax2 = plt.subplot(122)

ax2.set_xlabel(r'Abunds ratio')
ax2.set_ylim([40,110])
ax2.set_xlim([1.e-40,1.e2])
ax2.set_xscale('log')
line3, = ax2.plot([], [], label = r'Abunds ratio')
plt.legend()
plt.close()

In [5]:
def spike(x, C_spike):
    y = x-70
    if (y) == 0:
        return C_spike
    else:
        return 0
def spike_abs(N, C_spike):
    sp = np.zeros([2,len(N)])
    for i in range(len(N)):
        sp[0,i] = spike(N[i], C_spike)
        sp[1,i] = C_spike*np.heaviside(N[i]-69, 0)   
    return sp

Now, it's time to define a routine to draw a frame.

In [6]:
def drawframe(i, Z, s_array, delta, magic, t_9, Mu_n):
    T_9 = t_9[i]
    mu_p = -Mu_p[i]*Z**0
    result = np.zeros(len(Z))
    sp = sp_main(result, Z, s_array, delta, magic)
    intl= integral(Z, delta, magic) 
    txt_title.set_text(r"$T_9$ = {T_9} [MeV/k] ;  $-\mu_p'$ = {mu_p} [MeV]".format(T_9 = "%.1f" %T_9, mu_p = Mu_p[i]))
    
    f = ((s_array[1]*11.609)/(2*T_9)*((s_array[0] -intl[1,:]+ mu_p)/s_array[1]-1/2)**2)-((s_array[1]*11.609)/(2*T_9))*((Z-Z[0])-(s_array[0] -intl[1,:]+ mu_p)/s_array[1]+1/2)**2
    f1 = 11.609*(S_spike[1,:]+ intl[0,:]-intl[2,:]+s_array[2]*(Z-Z[0])*(Z-Z[0]+1)*(2*(Z-Z[0])+1)/6)/T_9

    fx = f+f1
    fx -= np.max(fx)
    F = math.e**(fx)
    
    line1.set_data(-mu_p, Z)
    line2.set_data(sp, Z)
    line3.set_data(F, Z)
    return (line1, line2, line3)

Now, let's input the proton number Z, s_array, delta and magic numbers. Another two important indexs are the temprature($T_9 \rm[MeV/k]$) and chemical potential $\mu_p' \rm[MeV]$, which will change the plot significantly. Simply speaking, if $T_9$ is fixed, when the $\mu_p'$ value is equal to $S_p$, the intersection of these two curves is the location of the abundance peak. On the other hand, if the value of $\mu_p'$ is fixed, with a decreasing $T_9$, you will see a narrowing Gaussian distribution. Of course, we can change both parametes at the same time.

In [11]:
s_array = [18, 0.25, 0.001]
delta = 1
magic = [(1.5, 50), (1.5, 82)]
t_9 = 1*np.ones(20)  
Mu_p = np.arange(20,0, -1)
C_spike = 0
Z = np.arange(40,110)
S_spike = np.array(spike_abs(Z, C_spike))
#t_9 = np.linspace(2,0.1,20) #use  to get a decreasing T
#Mu_p = 13*np.ones(20) #use  to get a fixed mu_n

Now, create the movie.

In [12]:
args = (Z, s_array, delta, magic, t_9, Mu_p)
anim = animation.FuncAnimation(fig, drawframe, fargs = args, frames=len(t_9), blit=True, repeat=False)
HTML(anim.to_jshtml())